In [37]:
from webscrapeData import updateData

updateData('2024-25')

updated Data in ../database/merged.csv
2024-25


In [38]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error as MSE

In [39]:
df = pd.read_csv('../database/merged.csv')
df['SALARY'] = df['SALARY'].replace('[\$,]', '', regex=True).astype(float)

df.dropna(subset='SALARY',inplace=True)
X = df.drop('SALARY', axis=1)
y = df['SALARY']
y = y.replace('[\$,]', '', regex=True).astype(float)




In [40]:
df.head()
df['PPG'] = round(df['PTS']/df['GP'],1)
df['RPG'] = round(df['REB']/df['GP'],1)
df['APG'] = round(df['AST']/df['GP'],1)



Feature Importance

In [41]:
columns_to_drop = ['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION']
X.drop(columns=[col for col in columns_to_drop if col in X.columns], inplace=True)

df_corr = pd.concat([X, y], axis=1)

# Compute Spearman correlation
spearman_corr = df_corr.corr(method='spearman')

# # Show correlation between each feature and the target
# corr_df = spearman_corr['SALARY'].sort_values(ascending=False)
# corr_df
corr_df = spearman_corr[['SALARY']].reset_index().rename(columns={'index': 'Feature'})

corr_df['ABS_CORR'] = abs(corr_df['SALARY'])
corr_df.sort_values(by = 'ABS_CORR', ascending=False, inplace=True)
corr_df.head(30)

,Feature,SALARY,ABS_CORR
61,SALARY,1.000000,1.000000
25,PTS,0.623405,0.623405
55,PTS_RANK,-0.623405,0.623405
36,FGM_RANK,-0.622615,0.622615
6,FGM,0.622615,0.622615
37,FGA_RANK,-0.616314,0.616314
7,FGA,0.616314,0.616314
60,WNBA_FANTASY_PTS_RANK,-0.614987,0.614987
30,WNBA_FANTASY_PTS,0.614987,0.614987
27,NBA_FANTASY_PTS,0.610239,0.610239


In [42]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import numpy as np
import pandas as pd

# Extract Spearman correlations with 'SALARY' as a DataFrame for merging
corr_df = spearman_corr[['SALARY']].reset_index().rename(columns={'index': 'Feature'})
corr_df['ABS_CORR'] = abs(corr_df['SALARY'])
corr_df.sort_values(by='ABS_CORR', ascending=False, inplace=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize model with parameters
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

# Train
model.fit(X_train, y_train)

# Predict & evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(MSE(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

# Create importance DataFrame
importance_df = pd.DataFrame({
    'Feature': model.feature_names_in_,
    'Importance': model.feature_importances_
})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)

# Filter features with importance > 0.01 (optional)
importance_filtered = importance_df[importance_df['Importance'] > 0.01]

# Merge importance with Spearman correlations on 'Feature'
corr_imp_table = importance_df.merge(corr_df, on='Feature')
corr_imp_table.sort_values(by = ['Importance','ABS_CORR'],inplace=True, ascending=False)
# feat_cut_df = corr_imp_table[(corr_imp_table['Importance']>0.01) & (corr_imp_table['ABS_CORR']>0.5)]
feat_cut_df = corr_imp_table[corr_imp_table['Feature'].str.contains('RANK')]
feat_cut_list = feat_cut_list = feat_cut_df['Feature'].tolist()
print(feat_cut_list)



RMSE: 7936280.10
['PTS_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3_PCT_RANK', 'PLUS_MINUS_RANK', 'FT_PCT_RANK', 'FGM_RANK', 'WNBA_FANTASY_PTS_RANK', 'NBA_FANTASY_PTS_RANK', 'FTM_RANK', 'PFD_RANK', 'FTA_RANK', 'TOV_RANK', 'AST_RANK', 'DREB_RANK', 'BLKA_RANK', 'STL_RANK', 'REB_RANK', 'DD2_RANK', 'FG3A_RANK', 'FG3M_RANK', 'PF_RANK', 'BLK_RANK', 'OREB_RANK', 'W_RANK', 'GP_RANK', 'TD3_RANK', 'L_RANK']


In [43]:
# Initialize model with parameters
feat_cut_list.append('AGE')
feat_cut_list.append('GP')
model_cut = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

# Train
model_cut.fit(X_train[feat_cut_list], y_train)

# Predict & evaluate
y_pred = model_cut.predict(X_test[feat_cut_list])
rmse = np.sqrt(MSE(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")


RMSE: 8070018.25


In [44]:
df['PREDICTED_SALARY'] = model_cut.predict(df[feat_cut_list])
top_pred_df = df.sort_values(by = 'PREDICTED_SALARY',ascending=False)
top_pred_df.head(15)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SALARY,PPG,RPG,APG,PREDICTED_SALARY
501,201939,Stephen Curry,Stephen,1610612744,GSW,37.0,70,41,29,0.586,...,43,19,71,44,13,55761216.0,24.5,4.4,6.0,54227932.0
277,203954,Joel Embiid,Joel,1610612755,PHI,31.0,19,8,11,0.421,...,390,289,87,20,287,51415938.0,23.8,8.2,4.5,50776088.0
424,203999,Nikola Jokić,Nikola,1610612743,DEN,30.0,70,46,24,0.657,...,2,1,2,1,1,51415938.0,29.6,12.7,10.2,50504952.0
330,201142,Kevin Durant,Kevin,1610612756,PHX,36.0,62,33,29,0.532,...,361,30,106,44,24,51179021.0,26.6,6.0,4.2,49840788.0
180,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,30.0,67,40,27,0.597,...,28,3,5,2,4,48787676.0,30.4,11.9,6.5,48819668.0
314,1626157,Karl-Anthony Towns,Karl-Anthony,1610612752,NYK,29.0,72,46,26,0.639,...,21,8,4,20,9,49205800.0,24.4,12.8,3.1,48718960.0
359,2544,LeBron James,LeBron,1610612747,LAL,40.0,70,44,26,0.629,...,383,10,17,3,10,48728845.0,24.4,7.8,8.2,47911904.0
141,1626164,Devin Booker,Devin,1610612756,PHX,28.0,75,35,40,0.467,...,507,14,35,44,11,49205800.0,25.6,4.1,7.1,47777356.0
255,1627759,Jaylen Brown,Jaylen,1610612738,BOS,28.0,63,46,17,0.730,...,18,54,106,44,52,49700000.0,22.2,5.8,4.5,46129724.0
451,202331,Paul George,Paul,1610612755,PHI,35.0,41,15,26,0.366,...,432,172,134,44,175,49205800.0,16.2,5.3,4.3,46124696.0


In [45]:
df['SALARY_DIFF'] = df['PREDICTED_SALARY'] - df['SALARY']
df['SALARY_PCT_CHANGE'] = ((df['PREDICTED_SALARY'] - df['SALARY'])/df['SALARY'])*100
# improved_df = df[df['SALARY_DIFF']>0]
# improved_df.sort_values(by = 'SALARY_DIFF',ascending=False)

df.sort_values(by = 'SALARY_DIFF',ascending=True)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SALARY,PPG,RPG,APG,PREDICTED_SALARY,SALARY_DIFF,SALARY_PCT_CHANGE
41,1627732,Ben Simmons,Ben,1610612746,LAC,28.0,51,26,25,0.510,...,198,44,261,40338144.0,5.0,4.7,5.6,13770520.0,-26567624.0,-65.862287
112,203081,Damian Lillard,Damian,1610612749,MIL,34.0,58,32,26,0.552,...,40,15,41,48787676.0,24.9,4.7,7.1,29977130.0,-18810546.0,-38.555938
316,202695,Kawhi Leonard,Kawhi,1610612746,LAC,33.0,37,26,11,0.703,...,106,44,169,49205800.0,21.5,5.9,3.1,31638676.0,-17567124.0,-35.701328
546,1630169,Tyrese Haliburton,Tyrese,1610612754,IND,25.0,73,46,27,0.630,...,18,44,19,42176400.0,18.6,3.5,9.2,26121964.0,-16054436.0,-38.064975
63,1628971,Bruce Brown,Bruce,1610612740,NOP,28.0,41,15,26,0.366,...,281,44,299,23000000.0,8.3,4.0,2.0,8550043.0,-14449957.0,-62.825900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,1630208,Nick Richards,Nick,1610612756,PHX,27.0,57,17,40,0.298,...,35,44,203,5000000.0,9.3,8.2,0.9,18421374.0,13421374.0,268.427480
185,1627824,Guerschon Yabusele,Guerschon,1610612755,PHI,29.0,70,23,47,0.329,...,134,44,122,2087519.0,11.0,5.6,2.1,17821566.0,15734047.0,753.719942
204,1627826,Ivica Zubac,Ivica,1610612746,LAC,28.0,80,50,30,0.625,...,2,20,17,11743210.0,16.8,12.6,2.7,30842712.0,19099502.0,162.642940
498,203915,Spencer Dinwiddie,Spencer,1610612742,DAL,32.0,79,37,42,0.468,...,106,44,108,2087519.0,11.0,2.6,4.4,22147400.0,20059881.0,960.943637


In [ ]:
import sqlite3

# Step 1: Create a connection to a SQLite database file (it creates the file if it doesn't exist)
# conn = sqlite3.connect('nba_data.db')
conn = sqlite3.connect('..\app\my-project\backend\nba_data.db')

# Step 2: Write the DataFrame to a SQL table
df.to_sql('player_data', conn, if_exists='replace', index=False)

# Step 3: (Optional) Close the connection when done
conn.close()


OperationalError: unable to open database file